<i>Copyright (c) Microsoft Corporation. All rights reserved.</i>

<i>Licensed under the MIT License.</i>

# xDeepFM : the eXtreme Deep Factorization Machine 
This notebook will give you a quick example of how to train an [xDeepFM model](https://arxiv.org/abs/1803.05170). 
xDeepFM \[1\] is a deep learning-based model aims at capturing both lower- and higher-order feature interactions for precise recommender systems. Thus it can learn feature interactions more effectively and manual feature engineering effort can be substantially reduced. To summarize, xDeepFM has the following key properties:
* It contains a component, named CIN, that learns feature interactions in an explicit fashion and in vector-wise level;
* It contains a traditional DNN component that learns feature interactions in an implicit fashion and in bit-wise level.
* The implementation makes this model quite configurable. We can enable different subsets of components by setting hyperparameters like `use_Linear_part`, `use_FM_part`, `use_CIN_part`, and `use_DNN_part`. For example, by enabling only the `use_Linear_part` and `use_FM_part`, we can get a classical FM model.

In this notebook, we test xDeepFM on two datasets: 1) a small synthetic dataset and 2) [Criteo dataset](http://labs.criteo.com/category/dataset)

## 0. Global Settings and Imports

In [1]:
import sys
sys.path.append("../../")
import os
import papermill as pm
from tempfile import TemporaryDirectory

import tensorflow as tf

from reco_utils.common.constants import SEED
from reco_utils.recommender.deeprec.deeprec_utils import (
    download_deeprec_resources, prepare_hparams
)
from reco_utils.recommender.deeprec.models.xDeepFM import XDeepFMModel
from reco_utils.recommender.deeprec.IO.iterator import FFMTextIterator

print("System version: {}".format(sys.version))
print("Tensorflow version: {}".format(tf.__version__))

tmpdir = TemporaryDirectory()

System version: 3.6.8 |Anaconda, Inc.| (default, Dec 30 2018, 01:22:34) 
[GCC 7.3.0]
Tensorflow version: 1.12.0


#### Parameters

In [2]:
EPOCHS_FOR_SYNTHETIC_RUN = 15
EPOCHS_FOR_CRITEO_RUN = 30
BATCH_SIZE_SYNTHETIC = 128
BATCH_SIZE_CRITEO = 4096
RANDOM_SEED = SEED  # Set None for non-deterministic result

xDeepFM uses the FFM format as data input: `<label> <field_id>:<feature_id>:<feature_value>`  
Each line represents an instance, `<label>` is a binary value with 1 meaning positive instance and 0 meaning negative instance. 
Features are divided into fields. For example, user's gender is a field, it contains three possible values, i.e. male, female and unknown. Occupation can be another field, which contains many more possible values than the gender field. Both field index and feature index are starting from 1. <br>

## 1. Synthetic data
Now let's start with a small synthetic dataset. In this dataset, there are 10 fields, 1000 fefatures, and label is generated according to the result of a set of preset pair-wise feature interactions. 

In [3]:
data_path = tmpdir.name
yaml_file = os.path.join(data_path, r'xDeepFM.yaml')
train_file = os.path.join(data_path, r'synthetic_part_0')
valid_file = os.path.join(data_path, r'synthetic_part_1')
test_file = os.path.join(data_path, r'synthetic_part_2')
output_file = os.path.join(data_path, r'output.txt')

if not os.path.exists(yaml_file):
    download_deeprec_resources(r'https://recodatasets.blob.core.windows.net/deeprec/', data_path, 'xdeepfmresources.zip')


10.6MB [00:00, 31.2MB/s]                            


#### 1.1 Prepare hyper-parameters
prepare_hparams() will create a full set of hyper-parameters for model training, such as learning rate, feature number, and dropout ratio. We can put those parameters in a yaml file, or pass parameters as the function's parameters (which will overwrite yaml settings).

In [4]:
hparams = prepare_hparams(yaml_file, 
                          FEATURE_COUNT=1000, 
                          FIELD_COUNT=10, 
                          cross_l2=0.0001, 
                          embed_l2=0.0001, 
                          learning_rate=0.001, 
                          epochs=EPOCHS_FOR_SYNTHETIC_RUN,
                          batch_size=BATCH_SIZE_SYNTHETIC)
print(hparams)

[('DNN_FIELD_NUM', None), ('FEATURE_COUNT', 1000), ('FIELD_COUNT', 10), ('MODEL_DIR', None), ('PAIR_NUM', None), ('SUMMARIES_DIR', None), ('activation', ['relu', 'relu']), ('attention_activation', None), ('attention_dropout', 0.0), ('attention_layer_sizes', None), ('batch_size', 128), ('cross_activation', 'identity'), ('cross_l1', 0.0), ('cross_l2', 0.0001), ('cross_layer_sizes', [1]), ('cross_layers', None), ('data_format', 'ffm'), ('dim', 10), ('doc_size', None), ('dropout', [0.0, 0.0]), ('dtype', 32), ('embed_l1', 0.0), ('embed_l2', 0.0001), ('enable_BN', False), ('entityEmb_file', None), ('entity_dim', None), ('entity_embedding_method', None), ('entity_size', None), ('epochs', 15), ('fast_CIN_d', 0), ('filter_sizes', None), ('init_method', 'tnormal'), ('init_value', 0.3), ('is_clip_norm', 0), ('iterator_type', None), ('kg_file', None), ('kg_training_interval', 5), ('layer_l1', 0.0), ('layer_l2', 0.0001), ('layer_sizes', [100, 100]), ('learning_rate', 0.001), ('load_model_name', 'yo

#### 1.2 Create data loader
Designate a data iterator for the model. xDeepFM uses FFMTextIterator. 

In [5]:
input_creator = FFMTextIterator

#### 1.3 Create model
When both hyper-parameters and data iterator are ready, we can create a model:

In [6]:
model = XDeepFMModel(hparams, input_creator, seed=RANDOM_SEED)

## sometimes we don't want to train a model from scratch
## then we can load a pre-trained model like this: 
#model.load_model(r'your_model_path')

Add CIN part.


/data/anaconda/envs/reco_gpu/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:112: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Now let's see what is the model's performance at this point (without starting training):

In [7]:
print(model.run_eval(test_file))

{'auc': 0.5043, 'logloss': 0.7515}


AUC=0.5 is a state of random guess. We can see that before training, the model behaves like random guessing.

#### 1.4 Train model
Next we want to train the model on a training set, and check the performance on a validation dataset. Training the model is as simple as a function call:

In [8]:
model.fit(train_file, valid_file)

at epoch 1 train info: auc:0.5189, logloss:0.7006 eval info: auc:0.504, logloss:0.7042
at epoch 1 , train time: 3.0 eval time: 3.1
at epoch 2 train info: auc:0.5365, logloss:0.6919 eval info: auc:0.5066, logloss:0.6973
at epoch 2 , train time: 2.7 eval time: 3.1
at epoch 3 train info: auc:0.5552, logloss:0.6884 eval info: auc:0.5099, logloss:0.6953
at epoch 3 , train time: 2.8 eval time: 3.1
at epoch 4 train info: auc:0.5784, logloss:0.6848 eval info: auc:0.5147, logloss:0.6946
at epoch 4 , train time: 2.9 eval time: 3.1
at epoch 5 train info: auc:0.6104, logloss:0.6783 eval info: auc:0.523, logloss:0.6941
at epoch 5 , train time: 2.7 eval time: 3.2
at epoch 6 train info: auc:0.6582, logloss:0.6625 eval info: auc:0.5416, logloss:0.6929
at epoch 6 , train time: 2.7 eval time: 3.1
at epoch 7 train info: auc:0.7318, logloss:0.6204 eval info: auc:0.5916, logloss:0.6831
at epoch 7 , train time: 2.8 eval time: 3.1
at epoch 8 train info: auc:0.83, logloss:0.5231 eval info: auc:0.7024, logloss

#### 1.5 Evaluate model

Again, let's see what is the model's performance now (after training):

In [9]:
res_syn = model.run_eval(test_file)
print(res_syn)
pm.record("res_syn", res_syn)

{'auc': 0.9716, 'logloss': 0.2278}


/data/anaconda/envs/reco_gpu/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Function record is deprecated and will be removed in verison 1.0.0 (current version 0.19.0). Please see `scrapbook.glue` (nteract-scrapbook) as a replacement for this functionality.
  This is separate from the ipykernel package so we can avoid doing imports until


If we want to get the full prediction scores rather than evaluation metrics, we can do this:

In [10]:
model.predict(test_file, output_file)

## 2. Criteo data 

Now we have successfully launched an experiment on a synthetic dataset. Next let's try something on a real world dataset, which is a small sample from [Criteo dataset](http://labs.criteo.com/category/dataset). Criteo dataset is a well known industry benchmarking dataset for developing CTR prediction models and it's frequently adopted as evaluation dataset by research papers. The original dataset is too large for a lightweight demo, so we sample a small portion from it as a demo dataset.

In [11]:
print('demo with Criteo dataset')
hparams = prepare_hparams(yaml_file, 
                          FEATURE_COUNT=2300000, 
                          FIELD_COUNT=39, 
                          cross_l2=0.01, 
                          embed_l2=0.01, 
                          layer_l2=0.01,
                          learning_rate=0.002, 
                          batch_size=BATCH_SIZE_CRITEO, 
                          epochs=EPOCHS_FOR_CRITEO_RUN, 
                          cross_layer_sizes=[20, 10], 
                          init_value=0.1, 
                          layer_sizes=[20,20],
                          use_Linear_part=True, 
                          use_CIN_part=True, 
                          use_DNN_part=True)

train_file = os.path.join(data_path, r'cretio_tiny_train')
valid_file = os.path.join(data_path, r'cretio_tiny_valid')
test_file = os.path.join(data_path, r'cretio_tiny_test')

demo with Criteo dataset


In [12]:
model = XDeepFMModel(hparams, FFMTextIterator, seed=RANDOM_SEED)

# check the predictive performance before the model is trained
print(model.run_eval(test_file)) 
model.fit(train_file, valid_file)
# check the predictive performance after the model is trained
res_real = model.run_eval(test_file)
print(res_real)
pm.record("res_real", res_real)

Add linear part.
Add CIN part.
Add DNN part.
{'auc': 0.4728, 'logloss': 0.7113}
at epoch 1 train info: auc:0.6648, logloss:0.5347 eval info: auc:0.6637, logloss:0.5342
at epoch 1 , train time: 27.9 eval time: 22.5
at epoch 2 train info: auc:0.7155, logloss:0.51 eval info: auc:0.7137, logloss:0.5109
at epoch 2 , train time: 24.3 eval time: 22.5
at epoch 3 train info: auc:0.7331, logloss:0.5012 eval info: auc:0.7283, logloss:0.5037
at epoch 3 , train time: 24.3 eval time: 22.2
at epoch 4 train info: auc:0.7412, logloss:0.4964 eval info: auc:0.7359, logloss:0.4991
at epoch 4 , train time: 24.1 eval time: 22.3
at epoch 5 train info: auc:0.7456, logloss:0.4937 eval info: auc:0.74, logloss:0.4963
at epoch 5 , train time: 24.1 eval time: 22.5
at epoch 6 train info: auc:0.7487, logloss:0.4919 eval info: auc:0.7426, logloss:0.4946
at epoch 6 , train time: 24.1 eval time: 22.3
at epoch 7 train info: auc:0.7505, logloss:0.4907 eval info: auc:0.7441, logloss:0.4934
at epoch 7 , train time: 24.2 ev

/data/anaconda/envs/reco_gpu/lib/python3.6/site-packages/ipykernel_launcher.py:9: DeprecationWarning: Function record is deprecated and will be removed in verison 1.0.0 (current version 0.19.0). Please see `scrapbook.glue` (nteract-scrapbook) as a replacement for this functionality.
  if __name__ == '__main__':


## Reference
\[1\] Lian, J., Zhou, X., Zhang, F., Chen, Z., Xie, X., & Sun, G. (2018). xDeepFM: Combining Explicit and Implicit Feature Interactions for Recommender Systems. Proceedings of the 24th ACM SIGKDD International Conference on Knowledge Discovery \& Data Mining, KDD 2018, London, UK, August 19-23, 2018.<br>

In [13]:
# Cleanup
tmpdir.cleanup()